[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/PO_II/blob/main/3_Otimização_Redes/Designação/designa_pyomo.ipynb)

# Pesquisa Operacional II

**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**

Monitores:
- Henrique Monteiro Soares da Silva (2023)

# Problema da Designação

Um Centro de Distribuição deve alocar quatro equipes de trabalho (recursos) a quatro reparos não iniciados.

A tabela abaixo apresenta os custos de designação:

In [ ]:
import pandas as pd

Vamos criar uma tabela de custos para o exemplo visto na aula!

In [ ]:
c = pd.DataFrame({'I':[6,8,3,3], 'II':[4,8,5,9], 'III':[3,10,8,8], 'IV':[5,7,6,10]}, index = ['A', 'B', 'C', 'D'])

Vamos imprimir nossa tabela na tela!

In [ ]:
c

In [ ]:
c_dict = c.stack().to_dict()

c_dict

In [ ]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!pip install -i https://pypi.gurobi.com gurobipy

In [ ]:
import pyomo.environ as pe

In [ ]:
M = pe.ConcreteModel()

In [ ]:
M.pprint()

In [ ]:
M.equipes = pe.Set(initialize = ['A', 'B', 'C', 'D'])
M.reparos = pe.Set(initialize = ['I', 'II', 'III', 'IV'])

In [ ]:
M.pprint()

In [ ]:
M.custos = pe.Param(M.equipes, M.reparos, initialize = c_dict)

In [ ]:
M.x = pe.Var(M.equipes, M.reparos, within= pe.NonNegativeReals)

In [ ]:
M.pprint()

In [ ]:
# R1: Cada equipe só pode ser alocada para um único reparo.
M.R_ofertas = pe.ConstraintList()
for i in M.equipes:
    M.R_ofertas.add(expr = sum(M.x[i, j] for j in M.reparos) == 1)


# R2: Cada centro de distribuição deve ter a sua demanda atendida
M.R_demandas = pe.ConstraintList()
for j in M.reparos:
    M.R_demandas.add(expr= sum(M.x[i, j] for i in M.equipes) == 1)

In [ ]:
M.pprint()

In [ ]:
def custo_total(M):
    return sum(M.x[i,j] * M.custos[i,j] for i in M.equipes for j in M.reparos)

# FUNÇÃO OBJETIVO
M.obj = pe.Objective(rule= custo_total ,sense= pe.minimize)
  

In [ ]:
pe.SolverFactory('gurobi').solve(M)


In [ ]:
print(f"z= {pe.value(M.obj)}")

for k in M.x.keys(): 
    if pe.value(M.x[k]) > 0:
        print (f"x_{k} = {pe.value(M.x[k])}")